In [1]:
import argparse
import json
from multiprocessing.sharedctypes import Value
import os,io,time
import pickle
import numpy as np
import sys

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import sys
# sys.path.insert(0,'/orcd/home/002/swans/relE_paper/relE_binder_design/TERMinator_sscore')
sys.path.insert(0,'/orcd/home/002/swans/relE_paper/TERMinator_sscore')
from scripts.data.preprocessing.cleanStructs import extractBackbone
from terminator.data.data import BinderScoringIterableDataset,ComplexScoringDataset
from terminator.models.TERMinator import TERMinator
from terminator.utils.model.loop_utils import run_epoch,_to_dev
from terminator.utils.model.loss_fn import construct_loss_fn
from terminator.utils.common import int_to_3lt_AA,AA_to_int,int_to_AA
from terminator.utils.model.default_hparams import DEFAULT_MODEL_HPARAMS, DEFAULT_TRAIN_HPARAMS

sys.path.insert(0,'/orcd/home/002/swans/relE_paper/relE_binder_design')
from src.rele_binder_design.score_utils import interfaceScorer,get_binder_names

/home/swans/.cache/pypoetry/virtualenvs/rele-binder-design-C4Q1T_FQ-py3.9/lib/python3.9/site-packages/torch/torch_version.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import packaging  # type: ignore[attr-defined]
/home/swans/.cache/pypoetry/virtualenvs/rele-binder-design-C4Q1T_FQ-py3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_hparams(model_path, default_hparams, output_name):
    print("loading params")
    # load hparams
    hparams_path = os.path.join(model_path, output_name)
    hparams = json.load(open(hparams_path, 'r'))
    for hparam in default_hparams:
        if hparam not in hparams:
            # print(f"{hparam} = {default_hparams[hparam]}")
            hparams[hparam] = default_hparams[hparam]
    return hparams

In [3]:
new = "{'cov_features': 'all_raw', 'cov_compress': 'project', 'term_use_mpnn': True, 'matches': 'transformer', 'energies_use_mpnn': True, 'energies_full_graph': True, 'contact_idx': True, 'energies_encoder_layers': 3, 'energies_hidden_dim': 128, 'resnet_linear': True, 'matches_linear': True, 'transformer_linear': True, 'term_mpnn_linear': True, 'use_terms': False, 'res_embed_linear': True, 'sscore_from_embedding': 'node', 'sscore_module': 'aggnode', 'model': 'multichain', 'term_hidden_dim': 32, 'flex_hidden_dim': 1, 'flex_type': '', 'gradient_checkpointing': True, 'num_pair_stats': 28, 'num_sing_stats': 0, 'resnet_blocks': 4, 'term_layers': 4, 'flex_layers': 4, 'term_heads': 4, 'conv_filter': 3, 'matches_layers': 4, 'matches_num_heads': 4, 'k_neighbors': 30, 'k_cutoff': None, 'cie_dropout': 0.1, 'cie_scaling': 500, 'cie_offset': 0, 'transformer_dropout': 0.1, 'energies_protein_features': 'full', 'energies_augment_eps': 0, 'energies_dropout': 0.1, 'energies_output_dim': 400, 'energies_gvp': False, 'energies_geometric': False, 'min_protein_len': 30, 'sscore_module_nhidden': 64, 'struct2seq_linear': False, 'use_flex': False, 'use_coords': True, 'num_features': 9, 'merge_pair_embeddings': True, 'fe_dropout': 0.1, 'fe_max_len': 1000, 'num_ensembles': 1}"
old = "{'cov_features': 'all_raw', 'cov_compress': 'project', 'term_use_mpnn': True, 'matches': 'transformer', 'energies_use_mpnn': True, 'energies_full_graph': True, 'contact_idx': True, 'energies_encoder_layers': 3, 'energies_hidden_dim': 128, 'resnet_linear': True, 'matches_linear': True, 'transformer_linear': True, 'term_mpnn_linear': True, 'use_terms': False, 'res_embed_linear': True, 'sscore_from_embedding': 'node', 'sscore_module': 'aggnode', 'model': 'multichain', 'term_hidden_dim': 32, 'flex_hidden_dim': 1, 'flex_type': '', 'gradient_checkpointing': True, 'num_pair_stats': 28, 'num_sing_stats': 0, 'resnet_blocks': 4, 'term_layers': 4, 'flex_layers': 4, 'term_heads': 4, 'conv_filter': 3, 'matches_layers': 4, 'matches_num_heads': 4, 'k_neighbors': 30, 'k_cutoff': None, 'cie_dropout': 0.1, 'cie_scaling': 500, 'cie_offset': 0, 'transformer_dropout': 0.1, 'energies_protein_features': 'full', 'energies_augment_eps': 0, 'energies_dropout': 0.1, 'energies_output_dim': 400, 'energies_gvp': False, 'energies_geometric': False, 'min_protein_len': 30, 'sscore_module_nhidden': 64, 'struct2seq_linear': False, 'use_flex': False, 'use_coords': True, 'num_features': 9, 'merge_pair_embeddings': True, 'fe_dropout': 0.1, 'fe_max_len': 1000, 'num_ensembles': 1}"
new == old

True

In [4]:
new = "Loaded custom peptide reference aa probabilites: {'A': 0.077239502, 'R': 0.064524195, 'N': 0.028576171, 'D': 0.050600675, 'C': 0.005304932, 'E': 0.093456877, 'Q': 0.056167561, 'G': 0.093409949, 'H': 0.024454938, 'I': 0.014555994, 'L': 0.050140503, 'K': 0.054443844, 'M': 0.015790823, 'F': 0.009907349, 'P': 0.136898625, 'S': 0.1183944, 'T': 0.061204935, 'W': 0.003948931, 'Y': 0.007683536, 'V': 0.033292756}"
old = "Loaded custom peptide reference aa probabilites: {'A': 0.077239502, 'R': 0.064524195, 'N': 0.028576171, 'D': 0.050600675, 'C': 0.005304932, 'E': 0.093456877, 'Q': 0.056167561, 'G': 0.093409949, 'H': 0.024454938, 'I': 0.014555994, 'L': 0.050140503, 'K': 0.054443844, 'M': 0.015790823, 'F': 0.009907349, 'P': 0.136898625, 'S': 0.1183944, 'T': 0.061204935, 'W': 0.003948931, 'Y': 0.007683536, 'V': 0.033292756}"
new == old

True

In [5]:
class Args(argparse.Namespace):
    dev = 'cpu'
    binder_dataset = '7_connectFragments_relBhelix-2seeds_memfixed_topologyfixed_fusedDB.pdb'
    binder_list = ''
    target_pdb = '../files/4FXE-ARG81-relax-noHyd_E_.pdb'
    model_dir = '/orcd/pool/008/swans/terminator_stuff/priority/230310_finetuneCOORDinator_sscore_mpnodeupdate'
    seq_mode = 'consensus_aa'
    score_mode = 'interface_only'
    custom_pep_reference = '/orcd/pool/008/swans/proteome_amino_acid_distribution/iupred_humanproteome0.7cutoff_aaprob.pkl'
    store=False
    
args=Args()
args

Args()

In [6]:
'''
If scoring many structures against a single target, use `BinderScoringIterableDataset` class. This
loads from a single multi-entry PDB file, where the first section is the target and the remaining 
sections are binder structures. A file of this format is written by "generateSeeds" in the interface Generator repo

If scoring complexes, provide a path to a file where each line is the path to a complex structure.
The name of the file must be of the format NAME_X_Y, where Y is the single character binder chain ID.
'''

dev = args.dev
if torch.cuda.device_count() == 0:
    dev = "cpu"

# Initialize the special dataloader for binders
binder_subset = get_binder_names(args.binder_list)
n_binders = len(binder_subset) if (binder_subset is not None) else "all"

# If a simple peptide reference state is being used, no need to load binder structures separately
load_mode = "complex_only" if args.custom_pep_reference != '' else "binder_and_complex"
# load_mode = 'binder_and_complex'
print(f"Will attempt to load {n_binders} total structures from the binder dataset in {load_mode} mode")
if (args.binder_dataset):
    # dataset = BinderScoringIterableDataset(args.binder_dataset,args.target_pdb)
    dataset = BinderScoringIterableDataset(filename=args.binder_dataset,target_pdb_path=args.target_pdb,max_res_num=27500,binder_subset=binder_subset,mode=load_mode,skip_package=False)
    dataset_iter = iter(dataset)
elif (args.complex_dataset):
    dataset = ComplexScoringDataset(args.complex_dataset,targetChainID=args.target_chain_id,binderChainID=args.binder_chain_id,mode=load_mode)
    dataset_iter = iter(dataset)
else:
    raise ValueError("Must provide either --binder_dataset or --complex_dataset")
# dataloader = DataLoader(dataset,batch_size=1)

# Load the model configuration parameters
model_hparams = load_hparams(args.model_dir, DEFAULT_MODEL_HPARAMS, 'model_hparams.json')
run_hparams = load_hparams(args.model_dir, DEFAULT_TRAIN_HPARAMS, 'run_hparams.json')

# backwards compatibility
if "cov_features" not in model_hparams.keys():
    model_hparams["cov_features"] = False
if "term_use_mpnn" not in model_hparams.keys():
    model_hparams["term_use_mpnn"] = False
if "matches" not in model_hparams.keys():
    model_hparams["matches"] = "resnet"
if "struct2seq_linear" not in model_hparams.keys():
    model_hparams['struct2seq_linear'] = False
if "energies_gvp" not in model_hparams.keys():
    model_hparams['energies_gvp'] = False
if "num_sing_stats" not in model_hparams.keys():
    model_hparams['num_sing_stats'] = 0
if "num_pair_stats" not in model_hparams.keys():
    model_hparams['num_pair_stats'] = 0
if "contact_idx" not in model_hparams.keys():
    model_hparams['contact_idx'] = False
if "fe_dropout" not in model_hparams.keys():
    model_hparams['fe_dropout'] = 0.1
if "fe_max_len" not in model_hparams.keys():
    model_hparams['fe_max_len'] = 1000
if "cie_dropout" not in model_hparams.keys():
    model_hparams['cie_dropout'] = 0.1

if "num_ensembles" in run_hparams.keys():
    model_hparams['num_ensembles'] = run_hparams['num_ensembles']
else:
    run_hparams['num_ensembles'] = 1
    model_hparams['num_ensembles'] = 1

if "use_flex" not in model_hparams.keys():
    model_hparams["use_flex"] = False
    model_hparams["flex_type"] = ""

print(model_hparams)

# Initialize the model
terminator = TERMinator(hparams=model_hparams, device=dev)

# Load weights from the best checkpoint during training
best_checkpoint_state = torch.load(os.path.join(args.model_dir, 'net_best_checkpoint.pt'), map_location=dev)
best_checkpoint = best_checkpoint_state['state_dict']
terminator.load_state_dict(best_checkpoint)
terminator.to(dev)
binderSubsetNames = get_binder_names(args.binder_list)

custom_pep_ref_aa_probs = None
if args.custom_pep_reference != "":
    with open(args.custom_pep_reference,"rb") as file:
        custom_pep_ref_aa_probs = pickle.load(file)
    print(f"Loaded custom peptide reference aa probabilites: {custom_pep_ref_aa_probs}")

if args.store:
    try:
        os.mkdir('output')
    except OSError as error:
        print(error)

# Run the model in eval mode, compute the loss and generate energy tables
scorer = None
first = True
if (args.binder_dataset):
    print('score binders against a single target: ',args.score_mode)
    start = stop = 0
    for packaged_binder_complex_data in dataset_iter:
        start = time.time()
        if scorer == None:
            scorer = interfaceScorer(dataset.target_packaged_data,packaged_binder_complex_data,terminator,custom_pep_ref_aa_probs,dev)
        else:
            scorer.set_new_binder_and_complex(packaged_binder_complex_data)
        scorer.get_binder_scores(args.seq_mode,args.score_mode)
        scorer.write_structure_scores(args.score_mode)
        scorer.write_residue_scores(args.score_mode)
        scorer.write_pep_seqs()
        # scorer.write_pep_seq_probs()
        stop = time.time()
        if args.store:
            if first:
                # only need to store once
                scorer.pickle_network_output_target('output/')
                first = False
            scorer.pickle_network_output_binder('output/')
            scorer.pickle_network_output_complex('output/')
        print('Elapsed time:',stop-start)
else:
    # Score complexes
    print('score complexes: ',args.score_mode)
    start = stop = 0
    for idx,(packaged_target_data,packaged_binder_complex_data) in enumerate(dataset_iter):
        start = time.time()
        if scorer == None:
            scorer = interfaceScorer(packaged_target_data,packaged_binder_complex_data,terminator,custom_pep_ref_aa_probs,dev)
        else:
            scorer.set_new_target(packaged_target_data)
            scorer.set_new_binder_and_complex(packaged_binder_complex_data)
        if binderSubsetNames is not None and scorer.get_pdb_name() not in binderSubsetNames:
            continue
        # else:
        #     print(f"scoring binder {scorer.get_pdb_name(idx)}")
        scorer.get_binder_scores(args.seq_mode,args.score_mode)
        scorer.write_structure_scores(args.score_mode)
        scorer.write_residue_scores(args.score_mode)
        scorer.write_pep_seqs()
        scorer.write_pep_seq_probs()
        stop = time.time()
        if args.store:
            scorer.pickle_network_output_target('output/')
            scorer.pickle_network_output_binder('output/')
            scorer.pickle_network_output_complex('output/')
        stop = time.time()
        print('Elapsed time:',stop-start)

scorer.close_files()

print('Done!')


Will attempt to load all total structures from the binder dataset in complex_only mode
Done packing tensor
No target structure found at the beginning of the file, instead will be reading from:  ../files/4FXE-ARG81-relax-noHyd_E_.pdb
Done packing tensor
loading params
loading params
{'cov_features': 'all_raw', 'cov_compress': 'project', 'term_use_mpnn': True, 'matches': 'transformer', 'energies_use_mpnn': True, 'energies_full_graph': True, 'contact_idx': True, 'energies_encoder_layers': 3, 'energies_hidden_dim': 128, 'resnet_linear': True, 'matches_linear': True, 'transformer_linear': True, 'term_mpnn_linear': True, 'use_terms': False, 'res_embed_linear': True, 'sscore_from_embedding': 'node', 'sscore_module': 'aggnode', 'model': 'multichain', 'term_hidden_dim': 32, 'flex_hidden_dim': 1, 'flex_type': '', 'gradient_checkpointing': True, 'num_pair_stats': 28, 'num_sing_stats': 0, 'resnet_blocks': 4, 'term_layers': 4, 'flex_layers': 4, 'term_heads': 4, 'conv_filter': 3, 'matches_layers': 4

In [7]:
scorer.bound_structure_scores[0].shape

torch.Size([116])

In [8]:
scorer.bound_structure_scores[0].sum()

tensor(-31.7820)

In [9]:
scorer.unbound_structure_scores[0].shape

torch.Size([116])

In [10]:
scorer.unbound_structure_scores[0].sum()

tensor(-22.4803)

In [11]:
scorer.bound_structure_scores[0].sum() - scorer.unbound_structure_scores[0].sum()

tensor(-9.3017)